<p>
  <a href="https://colab.research.google.com/github/gogitguhan/hands-on-lab-neo4j-and-vertex-ai/blob/main/Lab%205%20-%20Graph%20Data%20Science/embedding.ipynb" target="_blank">
    <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
  </a>
</p>

# Install Prerequisites
First off, you'll also need to install a few packages.

In [ ]:
%pip install --quiet --upgrade graphdatascience
%pip install --quiet google-cloud-storage

# Working with Neo4j
You'll need to enter the credentials from your Neo4j instance below.  You can get these by running the command ":server connect" in the Neo4j Browser.  The default DB_USER and DB_NAME are always neo4j.

In [ ]:
# Enter these Inputs!
CONN_URI = "neo4j+s://xxxxxxx.databases.neo4j.io:7687" #@param {type:"string"}
DB_PASS = "Your_Neo4j_Password" #@param {type:"string"}

# You can leave this default
DB_USER = 'neo4j'

In [ ]:
from graphdatascience import GraphDataScience
gds = GraphDataScience(CONN_URI, auth=(DB_USER, DB_PASS), aura_ds=True)

We will start with a clean Graph Data Science (GDS). Let's drop the 'mygraph', if it exists.

In [ ]:
result = gds.run_cypher(
  """
    CALL gds.graph.drop('mygraph',false)
  """
)
display(result)

First we're going to create an in memory graph represtation of the data in Neo4j Graph Data Science (GDS).

In [ ]:
result = gds.run_cypher(
  """
    CALL gds.graph.project(
      'mygraph',
      ['Company', 'Manager', 'Holding'],
      {
          OWNS: {orientation: 'UNDIRECTED'},
          PARTOF: {orientation: 'UNDIRECTED'}
      }
    )
    YIELD
      graphName AS graph,
      relationshipProjection AS readProjection,
      nodeCount AS nodes,
      relationshipCount AS rels
  """
)
display(result)

Now, let's list the details of the graph to make sure the projection was created as we want.

In [ ]:
result = gds.run_cypher(
  """
    CALL gds.graph.list()
  """
)
display(result)

Now we can generate an embedding from that graph. This is a new feature we can use in our predictions. We're using FastRP, which is a more full featured and higher performance of Node2Vec. You can learn more about that [here](https://neo4j.com/docs/graph-data-science/current/algorithms/fastrp/).

There are a bunch of parameters we could adjust in this.  One of the most obvious is the embeddingDimension.  The documentation covers many more.

In [ ]:
result = gds.run_cypher(
  """
  CALL gds.fastRP.mutate('mygraph',{
    embeddingDimension: 16,
    randomSeed: 1,
    mutateProperty:'embedding'
  })
  """
)
display(result)

That creates an embedding for each node type.  However, we only want the embedding on the nodes of type holding.

We're going to take the embedding from our projection and write it to the holding nodes in the underlying database.

In [ ]:
result = gds.run_cypher(
  """
    CALL gds.graph.writeNodeProperties('mygraph', ['embedding'], ['Holding'])
    YIELD writeMillis
  """
)
display(result)

In [ ]:
result = gds.run_cypher(
  """
    MATCH (n:Holding) RETURN n
  """
)
display(result)

Note that this query will take 2-3 minutes to run as it's grabbing nearly half a million nodes along with all their properties and our new embedding.

In [ ]:
import pandas as pd
df = pd.DataFrame([dict(record.items()) for record in result['n']])
df

Note that the embedding row is an array. To make this dataset more consumable, we should flatten that out into multiple individual features: embedding_0, embedding_1, ... embedding_n.


In [ ]:
embeddings = pd.DataFrame(df['embedding'].values.tolist()).add_prefix("embedding_")
merged = df.drop(columns=['embedding']).merge(embeddings, left_index=True, right_index=True)
merged

Now that we have the data formatted properly, let's split it into a training and a testing set and write those to disk.

In [ ]:
df = merged

df['split']=df['reportCalendarOrQuarter']
df['split']=df['split'].replace(['03-31-2021', '06-30-2021', '09-30-2021'], ['TRAIN', 'VALIDATE', 'TEST'])

df = df.drop(columns=['reportCalendarOrQuarter'])

df.to_csv('embedding.csv', index=False)

# Authenticate your Google Cloud Account
Now let's write the file to Google Cloud Storage so we can use it in our model.  To do so, we must first authenticate.

Edit the variables below.  You can find the project ID in the Google Cloud Console.  The STORAGE_BUCKET is the name of a new bucket.  It must be globally unique.  It also needs to be all lower case.

In [ ]:
# Edit this variable!
PROJECT_ID = 'your-project-id'

# You can leave these defaults
STORAGE_BUCKET = PROJECT_ID + '-form13'
REGION = 'us-central1'

In [ ]:
import os
os.environ['GCLOUD_PROJECT'] = PROJECT_ID

In [ ]:
try:
    from google.colab import auth as google_auth
    google_auth.authenticate_user()
except:
    pass

# Upload to Google Cloud Storage
Now we can upload our data sets to our bucket.

In [ ]:
from google.cloud import storage
client = storage.Client()

In [ ]:
bucket = client.bucket(STORAGE_BUCKET)
bucket.location=REGION
client.create_bucket(bucket)

In [ ]:
filename='embedding.csv'
upload_path = os.path.join('form13', filename)
blob = bucket.blob(upload_path)
blob.upload_from_filename(filename)